In [ ]:
# Define Cloudberry configuration

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')


In [ ]:
# Initialize API for data upload
cb_uploader = cb.AgeFileUploader(cb_config)

files_paths = [
    "./data/emas-20190411T232808.log",
    "./data/emas-20190411T234810.log",
    "./data/emas-20190412T000813.log",
    "./data/emas-20190412T120536.log",
]

files_details = cb.AgeUploadDetails(
    headers_keys={"[WH]": "[W]", "[SH]": "[S]", "[BH]": "[B]"}
)

experiment_name = 'Example AgE experiment'

uploaded_computations = []
for fp in files_paths:
    computation = cb_uploader.upload_file(file_path=fp, experiment_name=experiment_name, details=files_paths)
    uploaded_computations.append(computation)

In [ ]:
# Cleanup